## Build A simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with Langchain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting and an LLM call! 

After this I'll have a high level overview of:
- Using Language Models
- Using PromptTemplates and OutputParsers
- Using Langchain Expression Language(LCEL) to chain components together
- Debugging and tracing your application using LangSmith
- Deploying your application with LangServe

In [2]:
### Perplexity AI API key and Opem Source models --->Llama3, Gemma3, mistral ---> Groq
import os
from dotenv import load_dotenv

load_dotenv()

import openai 
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

model =ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025841F805F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025841E6BE60>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages =[
    SystemMessage(content= "Translate the following English text to French"),
    HumanMessage(content="Hello, How are you?")
]

result = model.invoke(messages)

In [6]:
result

AIMessage(content='Bonjour, comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 48, 'total_tokens': 56, 'completion_time': 0.016272322, 'prompt_time': 0.006149664, 'queue_time': 0.046149286, 'total_time': 0.022421986}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c40956ddc4', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--baf51e38-a2d4-4c81-85a8-094f84c6490e-0', usage_metadata={'input_tokens': 48, 'output_tokens': 8, 'total_tokens': 56})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ?'

In [9]:
### Using LCEL we can chain the components together
chain = model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generic_template), ("user", "{text}")
    ]
)

In [12]:
result = prompt.invoke({"language":"French", "text":"Hello"}) 

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [14]:
chain = prompt|model|parser
chain.invoke({"language":"French", "text":"Hello"})

'Bonjour'